In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import shutil
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import json

# Dataset
Se está utilizando el dataset "Recyclable and Household Waste Classification Dataset" obtenido de Kaggle.

Link: https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alistairking/recyclable-and-household-waste-classification")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'recyclable-and-household-waste-classification' dataset.
Path to dataset files: /kaggle/input/recyclable-and-household-waste-classification


In [ ]:
BASE_PATH = "/kaggle/input/recyclable-and-household-waste-classification/images/images"


# Preparación del dataset
El dataset original contiene 15000 imágenes organizadas uniformemente en 30 clases. Se decidió reorganizar las carpetas de forma que queán en las siguientes 6 clases:
- biodegradable
- glass
- metal
- non_recyclable
- paper
- plastic

In [ ]:
# Diccionario de clases

CLASS_MAP = {
    "biodegradable": [
        "coffee_grounds",
        "eggshells",
        "food_waste",
        "tea_bags"
    ],
    "plastic": [
        "disposable_plastic_cutlery",
        "plastic_cup_lids",
        "plastic_detergent_bottles",
        "plastic_food_containers",
        "plastic_shopping_bags",
        "plastic_soda_bottles",
        "plastic_straws",
        "plastic_trash_bags",
        "plastic_water_bottles"
    ],
    "paper": [
        "cardboard_boxes",
        "cardboard_packaging",
        "magazines",
        "newspaper",
        "office_paper",
        "paper_cups"
    ],
    "metal": [
        "aerosol_cans",
        "aluminum_food_cans",
        "aluminum_soda_cans",
        "steel_food_cans"
    ],
    "glass": [
        "glass_beverage_bottles",
        "glass_cosmetic_containers",
        "glass_food_jars"
    ],
    "non_recyclable": [
        "clothing",
        "shoes",
        "styrofoam_cups",
        "styrofoam_food_containers"
    ]
}


## Organización de carpetas

In [ ]:
OUTPUT_PATH = "dataset_clean"
# Crear estructura de salida para las carpetas train, val y test
for split in ["train", "val", "test"]:
    for cls in CLASS_MAP.keys():
        os.makedirs(os.path.join(OUTPUT_PATH, split, cls), exist_ok=True)

print("Generando dataset limpio...\n")

# Procesar cada categoría de los diccionarios
for final_class, original_classes in CLASS_MAP.items():
    print(f"Procesando categoría → {final_class}")

    all_images = []

    # Reunir imagenes de cada clase original
    for orig in original_classes:
        orig_path = os.path.join(BASE_PATH, orig)
        if not os.path.exists(orig_path):
            print(f"No existe: {orig_path}")
            continue

        # leer default/ y real_world/
        for sub in ["default", "real_world"]:
            subfolder = os.path.join(orig_path, sub)
            if not os.path.exists(subfolder):
                continue

            files = [
                os.path.join(subfolder, f)
                for f in os.listdir(subfolder)
                if f.endswith(".png")
            ]

            all_images.extend(files)

    print(f"  → Total imágenes encontradas: {len(all_images)}")

    # Dividir en train/val/test
    train_files, temp = train_test_split(all_images, test_size=0.3, random_state=42)
    val_files, test_files = train_test_split(temp, test_size=0.5, random_state=42)

    # Copiar archivos
    for f in train_files:
        shutil.copy(f, os.path.join(OUTPUT_PATH, "train", final_class))

    for f in val_files:
        shutil.copy(f, os.path.join(OUTPUT_PATH, "val", final_class))

    for f in test_files:
        shutil.copy(f, os.path.join(OUTPUT_PATH, "test", final_class))

print("\n✔ Dataset preparado en:", OUTPUT_PATH)

Generando dataset limpio...

Procesando categoría → biodegradable
  → Total imágenes encontradas: 2000
Procesando categoría → plastic
  → Total imágenes encontradas: 4500
Procesando categoría → paper
  → Total imágenes encontradas: 3000
Procesando categoría → metal
  → Total imágenes encontradas: 2000
Procesando categoría → glass
  → Total imágenes encontradas: 1500
Procesando categoría → non_recyclable
  → Total imágenes encontradas: 2000

✔ Dataset preparado en: dataset_clean


## Total de imágenes en la carpeta test

In [ ]:
DATASET_PATH = "dataset_clean/test"

# Obtener todas las clases
classes = sorted(os.listdir(DATASET_PATH))
print("Clases detectadas:", classes)

# Contar imágenes por clase
class_counts = {}
for cls in classes:
    path = os.path.join(DATASET_PATH, cls)
    count = len(os.listdir(path))
    class_counts[cls] = count
    print(f"{cls}: {count} imágenes")

# Crear etiquetas repetidas según frecuencia
labels = []
for idx, cls in enumerate(classes):
    labels.extend([idx] * class_counts[cls])
labels = np.array(labels)

Clases detectadas: ['biodegradable', 'glass', 'metal', 'non_recyclable', 'paper', 'plastic']
biodegradable: 177 imágenes
glass: 153 imágenes
metal: 177 imágenes
non_recyclable: 177 imágenes
paper: 219 imágenes
plastic: 236 imágenes


## Pesos por clase

In [ ]:
# Crear etiquetas según frecuencia
labels = []
for idx, cls in enumerate(classes):
    labels.extend([idx] * class_counts[cls])
labels = np.array(labels)

# Calcular class weights
weights = compute_class_weight(class_weight="balanced",
                               classes=np.arange(len(classes)),
                               y=labels)
class_weights = {i: w for i, w in enumerate(weights)}

print("\nClass weights calculados:")
for cls_idx, weight in class_weights.items():
    print(f"{classes[cls_idx]}: {weight:.2f}")

# Guardar class weights para entrenamiento
with open("class_weights.json", "w") as f:
    json.dump(class_weights, f)
print("\n✔ Class weights guardados en 'class_weights.json'")



Class weights calculados:
biodegradable: 1.00
glass: 1.00
metal: 1.00
non_recyclable: 1.00
paper: 1.00
plastic: 1.00

✔ Class weights guardados en 'class_weights.json'


El dataset está balanceado y cada clase contiene un número similar de muestras. Por ello, no es necesario hacer cambios adicionales para compensar desbalances.

# Entrenamiento

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
# Rutas
DATASET_PATH = "dataset_clean"
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20

In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input

# augmentation para train
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

# SOLO preprocess para val/test
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [ ]:
# Generadores
train_ds = train_datagen.flow_from_directory(
    DATASET_PATH + "/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_ds = val_datagen.flow_from_directory(
    DATASET_PATH + "/val",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_ds = test_datagen.flow_from_directory(
    DATASET_PATH + "/test",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


Found 1500 images belonging to 6 classes.
Found 1165 images belonging to 6 classes.
Found 1139 images belonging to 6 classes.


In [ ]:
# Modelo base
base_model = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False

In [ ]:
# Cabeza del modelo
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation="relu")(x)
output = Dense(6, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=output)

# Entrenamiento

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)


# compilar (fase 1)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Entrenamiento
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint]
)

# Guardar modelo
model.save("/content/drive/MyDrive/PC3/PC3/modelo_2/efficientnet_waste.h5")

Epoch 1/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 648ms/step - accuracy: 0.4901 - loss: 1.3916

47/47 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.4932 - loss: 1.3855 - val_accuracy: 0.7708 - val_loss: 0.6981
Epoch 2/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step - accuracy: 0.8136 - loss: 0.6106

47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 507ms/step - accuracy: 0.8138 - loss: 0.6098 - val_accuracy: 0.8240 - val_loss: 0.5223
Epoch 3/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step - accuracy: 0.8764 - loss: 0.4136

47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 514ms/step - accuracy: 0.8762 - loss: 0.4136 - val_accuracy: 0.8455 - val_loss: 0.4552
Epoch 4/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step - accuracy: 0.9001 - loss: 0.3462

47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 505ms/step - accuracy: 0.9001 - loss: 0.3459 - val_accuracy: 0.8584 - val_loss: 0.4115
Epoch 5/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 503ms/step - accuracy: 0.9164 - loss: 0.2904 - val_accuracy: 0.8532 - val_loss: 0.4052
Epoch 6/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step - accuracy: 0.9359 - loss: 0.2461

...

47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 507ms/step - accuracy: 0.9607 - loss: 0.1892 - val_accuracy: 0.8670 - val_loss: 0.3772
Epoch 9/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 23s 499ms/step - accuracy: 0.9675 - loss: 0.1437 - val_accuracy: 0.8644 - val_loss: 0.3764
Epoch 10/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step - accuracy: 0.9727 - loss: 0.1368

47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 512ms/step - accuracy: 0.9726 - loss: 0.1368 - val_accuracy: 0.8687 - val_loss: 0.3710
Epoch 11/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 505ms/step - accuracy: 0.9771 - loss: 0.1211 - val_accuracy: 0.8635 - val_loss: 0.3771
Epoch 12/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step - accuracy: 0.9803 - loss: 0.1112

47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 509ms/step - accuracy: 0.9802 - loss: 0.1112 - val_accuracy: 0.8747 - val_loss: 0.3723
Epoch 13/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 507ms/step - accuracy: 0.9804 - loss: 0.0925 - val_accuracy: 0.8730 - val_loss: 0.3690
Epoch 14/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 501ms/step - accuracy: 0.9842 - loss: 0.0817 - val_accuracy: 0.8738 - val_loss: 0.3611
Epoch 15/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 501ms/step - accuracy: 0.9836 - loss: 0.0813 - val_accuracy: 0.8678 - val_loss: 0.3708
Epoch 16/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 23s 496ms/step - accuracy: 0.9912 - loss: 0.0745 - val_accuracy: 0.8721 - val_loss: 0.3732
Epoch 17/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 504ms/step - accuracy: 0.9868 - loss: 0.0673 - val_accuracy: 0.8747 - val_loss: 0.3691
Epoch 18/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step - accuracy: 0.9909 - loss: 0.0633

47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 514ms/step - accuracy: 0.9908 - loss: 0.0634 - val_accuracy: 0.8781 - val_loss: 0.3838
Epoch 19/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 438ms/step - accuracy: 0.9924 - loss: 0.0588

47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 512ms/step - accuracy: 0.9923 - loss: 0.0588 - val_accuracy: 0.8798 - val_loss: 0.3775
Epoch 20/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 510ms/step - accuracy: 0.9845 - loss: 0.0592 - val_accuracy: 0.8738 - val_loss: 0.3738


# Métricas

In [ ]:
# Evaluación en test
loss, accuracy = model.evaluate(test_ds)
print("Test Accuracy:", accuracy)
print("Test Loss:", loss)


36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - accuracy: 0.8927 - loss: 0.3646
Test Accuracy: 0.8665496110916138
Test Loss: 0.3966206908226013


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report


true_labels = test_ds.classes
class_names = list(test_ds.class_indices.keys())

# Predicciones
pred_probs = model.predict(test_ds)
pred_labels = np.argmax(pred_probs, axis=1)

# matriz de confusion
cm = confusion_matrix(true_labels, pred_labels)

print("=== Matriz de confusión ===")
print(cm)

# Reporte por clase
print("\n=== Accuracy por clase ===")
report = classification_report(true_labels, pred_labels, target_names=class_names)
print(report)


36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step
=== Matriz de confusión ===
[[162   1   1   1   7   5]
 [  1 134   5   0   2  11]
 [  4   8 159   1   1   4]
 [  2   0   2 154  11   8]
 [  6   0   5  14 189   5]
 [  6  16  11   5   9 189]]

=== Accuracy por clase ===
                precision    recall  f1-score   support

 biodegradable       0.90      0.92      0.91       177
         glass       0.84      0.88      0.86       153
         metal       0.87      0.90      0.88       177
non_recyclable       0.88      0.87      0.88       177
         paper       0.86      0.86      0.86       219
       plastic       0.85      0.80      0.83       236

      accuracy                           0.87      1139
     macro avg       0.87      0.87      0.87      1139
  weighted avg       0.87      0.87      0.87      1139

